## Betting Lines Scrape

This notebook's primary function is to scrape the website sportsbookreview.com for gambling lines (both totals and spreads) corresponding to each game in the 2014-2018 NBA seasons. I took the data previously collected from basketball-reference.com, and used the dates when games were played to query the website and collect each day's betting lines. I then put this historical data in a JSON file.

In [ ]:
import bs4
import csv
import sys
import json
import time
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

#### dataframe_loader: 

- returns a dataframe containing one years games from a list of jsons containing each teams games for the season

In [ ]:
def dataframe_loader(years_games):
    years_stats = []
    for game in years_games:
        with open(f'{game}') as g:
            years_stats.append(json.load(g))
    all_games_year = [team for game_list in years_stats for game in game_list for team in game]
    df_year = pd.DataFrame(all_games_year)
    return df_year

In [ ]:
gl_2014 = !ls ../data/archived_data/*_2014.json
gl_2015 = !ls ../data/archived_data/*_2015.json
gl_2016 = !ls ../data/archived_data/*_2016.json
gl_2017 = !ls ../data/archived_data/*_2017.json
gl_2018 = !ls ../data/archived_data/*_2018.json

In [ ]:
df_2014 = dataframe_loader(gl_2014)
df_2015 = dataframe_loader(gl_2015)
df_2016 = dataframe_loader(gl_2016)
df_2017 = dataframe_loader(gl_2017)
df_2018 = dataframe_loader(gl_2018)



In [ ]:
df_all = df_2014.append([df_2015, df_2016, df_2017, df_2018], ignore_index=True)

In [ ]:
def ymd(date):
    year = date[0:4]
    month = date[4:6]
    day = date[6:8]
    ymd = datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d")
    return ymd

In [ ]:
df_all['date'] = df_all[0].map(lambda x:ymd(x[:8]))

In [ ]:
df_all.head()

In [ ]:
date_list = [date for date in df_all['date'].unique()]

In [ ]:
len(date_list)

#### get_betting_lines: 
- Function takes in a date, queries sportsbookreview.com for the corresponding page, and returns the day's betting lines labeled by team

In [ ]:
date_list[0]

In [ ]:
pd.to_datetime(date_list[0]).date()

In [ ]:
def get_betting_lines(date):
    info_list = []
    betting_page = requests.get(f'https://www.sportsbookreview.com/betting-odds/nba-basketball/merged/?date={date}')
    print(f"getting betting data for {pd.to_datetime(date).date()}")
    time.sleep(2)
    betting_page = BeautifulSoup(betting_page.text, 'html.parser')
    teams_list = []
    for row in betting_page.find_all('div', {'class': 'eventLine-value'}):
        teams_list.append(row.text)
    betting_lines = []
    for item in betting_page.find_all('div', {'class': 'event-holder holder-complete'}):
        for line in item.find('div', {'class': 'el-div eventLine-book'}):
            betting_lines.append(line.text)
    betting_lines = [line.replace('\xa0', ' ') for line in betting_lines]
    
    date_list = [date for item in betting_lines]
    zipped_teams_lines = zip(date_list, teams_list, betting_lines)
    finished = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"saved betting data for {pd.to_datetime(date).date()} at {finished}")
    return list(zipped_teams_lines)

In [ ]:
all_lines = []
for date in date_list:
    all_lines.append(get_betting_lines(date))
    

In [ ]:
with open('../data/all_gambling_lines.json', 'w') as f:
    json.dump(all_lines, f)

In [ ]:
print("System and module version information:")
print("\n")
print(f'Python version: {sys.version_info}')
print(f'pandas version: {pd.__version__}')
print(f'Beautiful Soup version: {bs4.__version__}')
print(f'requests version: {requests.__version__}')
print(f'last updated: {datetime.now().strftime("%Y-%m-%d %H:%M")}')